In [ ]:
!pip uninstall -y matplotlib
!pip install matplotlib==3.5.3

In [ ]:
import os
os.environ["MPLBACKEND"] = "Agg"

import matplotlib
matplotlib.use('Agg')

import skimage
skimage.io.use_plugin('pil')

from skimage import io, img_as_float32, transform


In [ ]:
!pip install flask-ngrok
!pip install pyngrok

In [ ]:
!apt update
!apt install ffmpeg -y

In [ ]:
!pip install opencv-python basicsr realesrgan imageio
!pip install torch==1.13.1 torchvision==0.14.1 --index-url https://download.pytorch.org/whl/cu117
!pip install gfpgan==1.3.8
!pip install -U scikit-image

In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10

In [ ]:
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3

In [ ]:
!git clone -b dev https://github.com/OpenTalker/SadTalker.git
%cd SadTalker
!python3 -m pip install -r requirements.txt
!bash scripts/download_models.sh

In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1 --index-url https://download.pytorch.org/whl/cu117

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment

In [ ]:
from flask import Flask, request, send_file
from pyngrok import ngrok
import os
import subprocess

ngrok.set_auth_token('2zBytGMcVk9x9BMsHaTl8LHQCsk_4ruiCTWtjRUNjapivYdEd')
app = Flask(__name__)
public_url = ngrok.connect(5000)
print(f"🔗 Public URL: {public_url}")

@app.route('/generate', methods=['POST'])
def generate():
    image = request.files.get('image')
    audio = request.files.get('audio')

    if not image or not audio:
        return "Missing image or audio", 400

    image_dir = "/content/image"
    audio_dir = "/content/audio"
    result_dir = "/content/results"
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(audio_dir, exist_ok=True)
    os.makedirs(result_dir, exist_ok=True)

    image_path = f"{image_dir}/received_image.jpg"
    audio_path = f"{audio_dir}/received_audio.wav"
    output_video_path = f"{result_dir}/received_video.mp4"

    image.save(image_path)
    audio.save(audio_path)

    command = [
        "python3", "/content/SadTalker/inference.py",
        "--driven_audio", audio_path,
        "--source_image", image_path,
        "--result_dir", result_dir,
        "--enhancer", "gfpgan"
    ]

    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        return f"Inference failed: {e}", 500

    if not os.path.exists(output_video_path):
        return "Video not found", 500

    return send_file(output_video_path, mimetype="video/mp4", as_attachment=True)

app.run(port=5000)